# One Hot Encoding and Training with LightGBM 

- In this first attempt I perform One Hot Encoding to the categorical features and ordinal encoding for the 'size' variable. Later I will train a LightGBM regressor with default parameters to predict The Price target variable.

In [4]:
import pandas as pd

In [ ]:
# If using local environment
df = pd.read_csv('cat_backpack.csv')

- One hot encoding for non hierarchycal features.

In [7]:

# Select the categorical columns to be encoded
categorical_cols = ['brand', 'material', 'style', 'color']

# Apply one-hot encoding using get_dummies
encoded_data = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)

# Concatenate the encoded data with the original DataFrame
df = pd.concat([df, encoded_data], axis=1)

# Drop the original categorical columns
df.drop(categorical_cols, axis=1, inplace=True)

- Ordinal encoding for 'size' column.

In [8]:
# Create a dictionary to map size categories to numerical values
size_mapping = {
    'Small': 0,
    'Medium': 1,
    'Large': 2,
    'Unknown': 3  # Or you can assign it -1 or another distinct value
}

# Apply the mapping to the 'size' column
df['size_encoded'] = df['size'].map(size_mapping)

# Drop the original 'size' column (optional)
df.drop('size', axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  300000 non-null  int64  
 1   compartments        300000 non-null  int64  
 2   laptop_compartment  300000 non-null  bool   
 3   waterproof          300000 non-null  bool   
 4   weight_cap          300000 non-null  float64
 5   Price               300000 non-null  float64
 6   brand_Adidas        300000 non-null  bool   
 7   brand_Jansport      300000 non-null  bool   
 8   brand_Nike          300000 non-null  bool   
 9   brand_Puma          300000 non-null  bool   
 10  brand_Under Armour  300000 non-null  bool   
 11  brand_Unknown       300000 non-null  bool   
 12  material_Canvas     300000 non-null  bool   
 13  material_Leather    300000 non-null  bool   
 14  material_Nylon      300000 non-null  bool   
 15  material_Polyester  300000 non-nul

# Train with LigthGBM.

In [10]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Drop the id column
df = df.drop(columns=['id'])

# Separate features and target variable
X = df.drop(columns=['Price'])
y = df['Price']

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the scaler and fit on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create LightGBM datasets
train_data = lgb.Dataset(X_train_scaled, label=y_train)
test_data = lgb.Dataset(X_test_scaled, label=y_test, reference=train_data)

# Set up model parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'seed': 42
}

In [12]:
# Train the model
regressor = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)],  # Include log_evaluation callback
    # verbose_eval=100  # Remove this line
)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	valid_0's rmse: 38.9136


In [13]:
# Predictions and evaluation
y_pred = regressor.predict(X_test_scaled, num_iteration=regressor.best_iteration)
# Calculate RMSE without the 'squared' argument, then take the square root
mse = mean_squared_error(y_test, y_pred)  # Remove squared=False
rmse = mse**0.5  # Calculate the square root to get RMSE
print("Test RMSE:", rmse)

Test RMSE: 38.91359184477554


In [8]:
# Import necessary libraries
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

# Create a new instance of the LGBMRegressor with the best parameters
best_params = {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': -1,
              'min_child_samples': 30, 'n_estimators': 100, 'num_leaves': 31,
              'subsample': 0.6}
final_model = LGBMRegressor(**best_params, random_state=42)

# Fit the model to the training data (X_train_scaled, y_train)
final_model.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor(colsample_bytree=0.6, learning_rate=0.01, min_child_samples=30,
              random_state=42, subsample=0.6)

In [9]:
# Predict on the test data (X_test_scaled)
y_pred = final_model.predict(X_test_scaled)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [10]:
# Calculate the RMSE to evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print("Test RMSE with Best Parameters:", rmse)

Test RMSE with Best Parameters: 38.9218926963783


# Optuna with LightGBM

In [11]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [12]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'random_state': 42,
        'n_estimators': 100,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
    }

    model = lgb.LGBMRegressor(**params)
    # Pass early stopping as a callback
    model.fit(
        X_train_scaled,
        y_train,
        eval_set=[(X_test_scaled, y_test)],
        callbacks=[lgb.early_stopping(stopping_rounds=10)]
    )
    predictions = model.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, predictions)
    return rmse

In [13]:
import optuna
from optuna.samplers import TPESampler

sampler = TPESampler(seed=1) #  Uses the Tree-Structured Parzen Estimator algorithm [4]
study = optuna.create_study(direction='minimize', sampler = sampler)

[I 2025-02-11 16:06:38,880] A new study created in memory with name: no-name-49ccd584-4f8e-4fb1-afae-6b9a4bda91b3


In [14]:
import time
start_time = time.time()
study.optimize(objective, n_trials=100)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-11 16:06:40,539] Trial 0 finished with value: 1514.220771409521 and parameters: {'learning_rate': 0.047531980423231666, 'num_leaves': 78, 'max_depth': 3, 'min_child_samples': 74, 'subsample': 0.5587023563268452, 'colsample_bytree': 0.5369354379075191, 'reg_alpha': 1.8626021219141067, 'reg_lambda': 3.45560727697487}. Best is trial 0 with value: 1514.220771409521.


Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 38.913


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

Early stopping, best iteration is:
[72]	valid_0's rmse: 38.908


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-11 16:09:41,195] Trial 99 finished with value: 1514.3705318275065 and parameters: {'learning_rate': 0.08742309776645198, 'num_leaves': 69, 'max_depth': 10, 'min_child_samples': 138, 'subsample': 0.5610321539815638, 'colsample_bytree': 0.6836411403664809, 'reg_alpha': 4.406211014889559, 'reg_lambda': 5.434008713487626}. Best is trial 5 with value: 1513.7200049780752.


Early stopping, best iteration is:
[22]	valid_0's rmse: 38.9149
Time taken: 182.3079128265381 seconds


In [15]:
print('Best parameters:', study.best_params)

Best parameters: {'learning_rate': 0.09899749800158453, 'num_leaves': 80, 'max_depth': 5, 'min_child_samples': 162, 'subsample': 0.5412904026310568, 'colsample_bytree': 0.679157410470362, 'reg_alpha': 9.085955031845, 'reg_lambda': 2.936141490800653}


- first iteration:
Best parameters: {'learning_rate': 0.09495644380018639, 'num_leaves': 77, 'max_depth': 3, 'min_child_samples': 62, 'subsample': 0.6614807453665343, 'colsample_bytree': 0.6148551220728337, 'reg_alpha': 8.500355486635911, 'reg_lambda': 7.010097067336246}
- 2nd iteration:
Best parameters: {'learning_rate': 0.09654269565258655, 'num_leaves': 82, 'max_depth': 3, 'min_child_samples': 77, 'subsample': 0.8443805713604343, 'colsample_bytree': 0.5006284268817309, 'reg_alpha': 4.506548701012825, 'reg_lambda': 3.5879714165911607}
- 3rd
Best Parameters: {'learning_rate': 0.09899749800158453, 'num_leaves': 80, 'max_depth': 5, 'min_child_samples': 162, 'subsample': 0.5412904026310568, 'colsample_bytree': 0.679157410470362, 'reg_alpha': 9.085955031845, 'reg_lambda': 2.936141490800653}
- 4th iteration:
Best Parameters: {'learning_rate': 0.09689056850918965, 'num_leaves': 45, 'max_depth': 4, 'min_child_samples': 42, 'subsample': 0.7495256437518634, 'colsample_bytree': 0.528430956341894, 'reg_alpha': 9.438202535340269, 'reg_lambda': 5.180036018469718}


In [16]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Best parameters from Optuna
best_params = study.best_params
print("Best Parameters:", best_params)

# Create and train the model with the best parameters
model = lgb.LGBMRegressor(**best_params)
model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
          callbacks=[lgb.early_stopping(stopping_rounds=10)])

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print("Test RMSE with Best Parameters:", rmse)

Best Parameters: {'learning_rate': 0.09899749800158453, 'num_leaves': 80, 'max_depth': 5, 'min_child_samples': 162, 'subsample': 0.5412904026310568, 'colsample_bytree': 0.679157410470362, 'reg_alpha': 9.085955031845, 'reg_lambda': 2.936141490800653}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[34]	valid_0's l2: 1514.16
Test RMSE with Best Parameters: 38.912187498337296


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
